# Demo Widget Composition and Interaction

In [ ]:
(require '[clojupyter-plugin.widgets :as widget])
(require '[clojupyter-plugin.widgets.control :as ctl]);

## Composition
To combine widgets, we have two include them in a container widget.
e.g. **box**, **v-box**, **h-box**, **grid-box**, **tab**, **accordion** and **stacked**.

The widgets can be deeply nested

In [ ]:
(let [sl (widget/int-slider)
      b0 (widget/h-box {:children [sl sl]})
      ac (widget/accordion {:children [b0]})
      b1 (widget/v-box {:children (vec (repeat 5 ac))})
      tc (widget/tab {:children (vec (repeat 10 b1))})]
  tc)

All of them use the :children key to hold the relationships.

## Widgets Linking
We can link the attributes of two widgets with link or directional-link widgets.

These allow linking the widgets directly, without relying on the kernel to handle callbacks. Evaluate the form bellow and reset the kernel. The widgets will continue to sync.

In [ ]:
(let [sl (widget/int-slider)
      p (widget/int-progress)]
  (widget/directional-link {:source [sl :value] :target [p :value]})
  (widget/box {:children [sl p]}))

## Stateless Callbacks
Some widgets send on-click and on-submit callbacks which can be handled by attaching a fn of 3 arguments (the widget reference. the message content, and a sequence of buffers) to the `:callbacks` map `:on-click` and `:on-submit` respectively.

In [ ]:
(let [out (widget/label)
      out2 (widget/label)
      but (widget/button {:description "Click me" :callbacks {:on-click [(fn [_ _ _] (swap! out assoc :value "Button clicked"))
                                                                      (fn [_ _ _] (swap! out2 assoc :value "Second fn ran"))]}})]
  (widget/v-box {:children [but out out2]}))

In [ ]:
(def L0 (widget/label))
(def BUTS (widget/toggle-buttons {:options ["one" "two"] :callbacks {:on-click (fn [w _ _] (swap! L0 assoc :value (str "Toggle pressed on " (:value @w))))}}))
(widget/v-box {:children [L0 BUTS]})

In [ ]:
(def W1
  (let [OUT (widget/label)
        W (widget/combobox {:options ["blue" "black" "green" "yellow"] :description "Pick a color"
                             :callbacks {:on-submit (fn [_ _ _] (swap! OUT assoc :value "User submitted"))}})]
    (widget/v-box {:children [W OUT]})))
W1

## Statefull Callbacks
Any widget can attach a fn using the watch method to run whenever the widget state changes.
The fn must have the same signature as those accepted by atom watchers: four arguments, a key, the reference, the old-state and new-state.

*Note*: Often, we're only interested when a single attribute of the widget changes, so we'll have to check it ourselves.
The example bellow attaches two watch fns, one that watches the int slider and updates the image `:width` attribute and a second fn that watches the file-upload widget and updates the image `:value` attribute.

In [ ]:
(def W
(let [w0 (widget/file-upload {:description "Upload an image" :multiple false})
      w1 (widget/int-slider {:min 300 :max 1000 :value 400 :step 10})
      w2 (widget/image {:width (str (:value @w1))})
      ww (widget/h-box {:children [w0 w1]})
      _ (add-watch w1 :on-change (fn [_ _ o-state n-state] (when (not= (:value o-state) (:value n-state)) (swap! w2 assoc :width (str (:value n-state))))))
      _ (add-watch w0 :on-change (fn [_ _ o-state n-state] (when (not= (:data o-state) (:data n-state)) (swap! w2 assoc :value (-> n-state :data first)))))]
  (widget/v-box {:children [ww w2]})))W

In [ ]:
(bytes? (first (:data @(first (:children @(first (:children @W)))))))

We can access helper functions in `ipywidgets.alpha` namespace.
`interactive` accepts an output widget, a fn that accepts a map argument and a map of widgets.

In [ ]:
(ctl/interactive (widget/int-progress {:max 200}) (comp (partial * 2) :x) {:x (widget/int-slider {:max 100})})

In [ ]:
(def IA
    (let [myfun (fn [{:keys [a b]}]
                  (str a " + " b " = " (+ a b)))
          label (widget/label)
          slider-1 (widget/int-slider {:value (rand-int 100)})
          slider-2 (widget/int-slider {:value (rand-int 100)})]
      (ctl/interactive label myfun {:a slider-1, :b slider-2})))
IA

`interact!` accepts a fn and one or more widgets. The fn must accept the same number of arguments as passed widgets. The return value of the fn is passed to a string and into an label widget.

In [ ]:
(def IB (ctl/interact! -
                         (widget/int-slider {:value (rand-int 100)})
                         (widget/int-slider {:value (rand-int 100)})))
IB

The example bellow contains a horizontal box widget with three other widgets, two int sliders and an int progress bar and combines both watch functions and link widgets.

In [ ]:
(def SSP
    (let [slider-style (widget/slider-style {:handle_color "tomato"})
      w (widget/int-slider {:orientation "vertical" :value (rand-int 101) :style slider-style :description "x"})
      ww (widget/int-slider {:orientation "vertical" :value (* 2 (:value @w)) :max (* 2 (:max @w)) :style slider-style :description "2*x" :disabled true})
      _ (add-watch w :double (fn [_ _ _ {value :value}] (swap! ww assoc :value (* 2 value))))
      p (widget/int-progress {:orientation "vertical" :value (:value @w) :bar_style "danger"})
      pv (widget/label {:value (str (:value @p)) :_dom_classes ["output"]})
      _ (add-watch p :print (fn [_ _ _ {value :value}] (swap! pv assoc :value (str value))))
      _ (widget/directional-link {:source [w :value] :target [p :value]})
      ppv (widget/v-box {:children [p pv] :layout (widget/layout {:align_items "center"})})]
  (widget/h-box {:children [w ww ppv]})))
SSP

In [ ]:
(let [sl0 (widget/int-slider)
      sl1 (widget/int-slider)
      val (widget/label)
      out (widget/output)
      cl (widget/button {:callbacks {:on-click (fn [_ _ _] (swap! out update :outputs empty))} :description "Clear"})]
  (ctl/tie! (widget/capture out #(str (/ % (:value @sl1)))) sl0 val)
  (ctl/tie! (widget/capture out #(str (/ (:value @sl0) %))) sl1 val)
  (widget/h-box {:children [(widget/v-box {:children [sl0 sl1 val]})
                         (widget/v-box {:children [cl out]})]}))         